# Time Series Data - Deep Learning

In [4]:
import pandas as pd
import numpy as np
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
#import plotly.plotly as py
#import plotly.graph_objs as go
#from plotly.offline import init_notebook_mode, iplot
#init_notebook_mode(connected=True)

# Set seeds to make the experiment more reproducible.
from tensorflow import set_random_seed
from numpy.random import seed
set_random_seed(1)
seed(1)
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional

Using TensorFlow backend.


In [69]:
df_price = pd.read_csv('./폐렴(2014-2018).csv',parse_dates=['date'],index_col='date' ,encoding='cp949')

In [71]:
from sklearn.preprocessing import MinMaxScaler

In [72]:
scaler = MinMaxScaler()

In [73]:
df = df_price

In [74]:
scale_cols = []
#scale_cols = [']

In [75]:
df_scaled = scaler.fit_transform(df[scale_cols])

In [76]:
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols

In [78]:
TEST_SIZE = 
WINDOW_SIZE = 

In [ ]:
train = df_scaled[:-TEST_SIZE]

In [ ]:
test = df_scaled[-TEST_SIZE:]

In [81]:
def make_dataset(data, label, window_size = 14):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [82]:
feature_cols = []
#feature_cols = []
label_cols = []

In [83]:
train_feature = train[feature_cols]
train_label = train[label_cols]

In [84]:
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid = train_test_split(train_feature, train_label, test_size=0.1)

In [86]:
test_feature = test[feature_cols]
test_label = test[label_cols]

# LSTM for Time Series Forecasting

In [89]:
train_feature, train_label = make_dataset(train_feature, train_label, 14)

In [90]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.1)

In [92]:
test_feature = test[feature_cols]
test_label = test[label_cols]

In [94]:
test_feature, test_label = make_dataset(test_feature, test_label, 14)

In [97]:
model = Sequential()
model.add(LSTM(50, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               return_sequences=False)
          )
#model.add(Dropout(0.2))
# model.add(LSTM(16, return_sequences=False))
# model.add(Dense(16, activation='linear'))
model.add(Dense(1))

In [98]:
model_path = './'

In [99]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
model.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', patience=5)
filename = os.path.join(model_path, 'tmp_checkpoint.h5')
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit(x_train, y_train, 
                                    epochs=100, 
                                    batch_size=1,
                                    validation_data=(x_valid, y_valid), 
                                    callbacks=[early_stop, checkpoint])

Train on 1535 samples, validate on 171 samples
Epoch 1/100
1535/1535 [==============================] - 15s 10ms/step - loss: 0.0190 - val_loss: 0.0125

Epoch 00001: val_loss improved from inf to 0.01246, saving model to E:/프로젝트/중기청/20190215_텍스트마이닝 자체 회의/20190215_텍스트마이닝 자체 회의/뉴스 데이터 추출/분석/통합(2014-2018)/분석/LSTM/tmp_checkpoint.h5
Epoch 2/100
1535/1535 [==============================] - 14s 9ms/step - loss: 0.0124 - val_loss: 0.0187

Epoch 00002: val_loss did not improve from 0.01246
Epoch 3/100
1535/1535 [==============================] - 13s 8ms/step - loss: 0.0108 - val_loss: 0.0100

Epoch 00003: val_loss improved from 0.01246 to 0.00998, saving model to E:/프로젝트/중기청/20190215_텍스트마이닝 자체 회의/20190215_텍스트마이닝 자체 회의/뉴스 데이터 추출/분석/통합(2014-2018)/분석/LSTM/tmp_checkpoint.h5
Epoch 4/100
1535/1535 [==============================] - 13s 9ms/step - loss: 0.0097 - val_loss: 0.0097

Epoch 00004: val_loss improved from 0.00998 to 0.00971, saving model to E:/프로젝트/중기청/20190215_텍스트마이닝 자체 회의/20190215_텍스트마이닝 자

In [100]:
model.load_weights(filename)

In [101]:
pred = model.predict(test_feature)

# CNN for Time Series Forecasting

In [56]:
model_cnn = Sequential()
model_cnn.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(train_feature.shape[1], train_feature.shape[2])))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dense(50, activation='relu'))
#model_cnn.add(Dropout(0.2))
model_cnn.add(Dense(1))

In [57]:
model_path = './'

In [58]:
model_cnn.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', patience=5)
filename = os.path.join(model_path, 'tmp_checkpoint.h5')
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

cnn_history = model_cnn.fit(x_train, y_train, 
                                    epochs=100, 
                                    batch_size=1,
                                    validation_data=(x_valid, y_valid), 
                                    callbacks=[early_stop, checkpoint])

Train on 1535 samples, validate on 171 samples
Epoch 1/100
1535/1535 [==============================] - 3s 2ms/step - loss: 0.0296 - val_loss: 0.0208

Epoch 00001: val_loss improved from inf to 0.02082, saving model to E:/프로젝트/중기청/20190215_텍스트마이닝 자체 회의/20190215_텍스트마이닝 자체 회의/뉴스 데이터 추출/분석/통합(2014-2018)/분석/CNN/tmp_checkpoint.h5
Epoch 2/100
1535/1535 [==============================] - 3s 2ms/step - loss: 0.0174 - val_loss: 0.0163

Epoch 00002: val_loss improved from 0.02082 to 0.01631, saving model to E:/프로젝트/중기청/20190215_텍스트마이닝 자체 회의/20190215_텍스트마이닝 자체 회의/뉴스 데이터 추출/분석/통합(2014-2018)/분석/CNN/tmp_checkpoint.h5
Epoch 3/100
1535/1535 [==============================] - 3s 2ms/step - loss: 0.0118 - val_loss: 0.0144

Epoch 00003: val_loss improved from 0.01631 to 0.01445, saving model to E:/프로젝트/중기청/20190215_텍스트마이닝 자체 회의/20190215_텍스트마이닝 자체 회의/뉴스 데이터 추출/분석/통합(2014-2018)/분석/CNN/tmp_checkpoint.h5
Epoch 4/100
1535/1535 [==============================] - 3s 2ms/step - loss: 0.0085 - val_loss: 0.0095

E

In [59]:
model_cnn.load_weights(filename)

In [60]:
pred = model_cnn.predict(test_feature)

# GRU for Time Series Forecasting

In [63]:
regressorGRU = Sequential()
# First GRU layer with Dropout regularisation
regressorGRU.add(GRU(units=50, return_sequences=False, input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu'))
#regressorGRU.add(Dropout(0.2))
regressorGRU.add(Dense(1))

In [64]:
model_path = './'

In [65]:
regressorGRU.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', patience=5)
filename = os.path.join(model_path, 'tmp_checkpoint.h5')
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

gru_history = regressorGRU.fit(x_train, y_train, 
                                    epochs=100, 
                                    batch_size=1,
                                    validation_data=(x_valid, y_valid), 
                                    callbacks=[early_stop, checkpoint])

Train on 1535 samples, validate on 171 samples
Epoch 1/100
1535/1535 [==============================] - 10s 7ms/step - loss: 0.0294 - val_loss: 0.0255

Epoch 00001: val_loss improved from inf to 0.02546, saving model to E:/프로젝트/중기청/20190215_텍스트마이닝 자체 회의/20190215_텍스트마이닝 자체 회의/뉴스 데이터 추출/분석/통합(2014-2018)/분석/GRU/tmp_checkpoint.h5
Epoch 2/100
1535/1535 [==============================] - 10s 6ms/step - loss: 0.0228 - val_loss: 0.0220

Epoch 00002: val_loss improved from 0.02546 to 0.02202, saving model to E:/프로젝트/중기청/20190215_텍스트마이닝 자체 회의/20190215_텍스트마이닝 자체 회의/뉴스 데이터 추출/분석/통합(2014-2018)/분석/GRU/tmp_checkpoint.h5
Epoch 3/100
1535/1535 [==============================] - 10s 6ms/step - loss: 0.0194 - val_loss: 0.0196

Epoch 00003: val_loss improved from 0.02202 to 0.01959, saving model to E:/프로젝트/중기청/20190215_텍스트마이닝 자체 회의/20190215_텍스트마이닝 자체 회의/뉴스 데이터 추출/분석/통합(2014-2018)/분석/GRU/tmp_checkpoint.h5
Epoch 4/100
1535/1535 [==============================] - 10s 6ms/step - loss: 0.0169 - val_loss: 0.014

1535/1535 [==============================] - 10s 6ms/step - loss: 0.0031 - val_loss: 0.0039

Epoch 00035: val_loss did not improve from 0.00368
Epoch 36/100
1535/1535 [==============================] - 10s 6ms/step - loss: 0.0031 - val_loss: 0.0048

Epoch 00036: val_loss did not improve from 0.00368
Epoch 37/100
1535/1535 [==============================] - 10s 6ms/step - loss: 0.0030 - val_loss: 0.0033

Epoch 00037: val_loss improved from 0.00368 to 0.00334, saving model to E:/프로젝트/중기청/20190215_텍스트마이닝 자체 회의/20190215_텍스트마이닝 자체 회의/뉴스 데이터 추출/분석/통합(2014-2018)/분석/GRU/tmp_checkpoint.h5
Epoch 38/100
1535/1535 [==============================] - 10s 6ms/step - loss: 0.0028 - val_loss: 0.0047

Epoch 00038: val_loss did not improve from 0.00334
Epoch 39/100
1535/1535 [==============================] - 10s 6ms/step - loss: 0.0029 - val_loss: 0.0032

Epoch 00039: val_loss improved from 0.00334 to 0.00319, saving model to E:/프로젝트/중기청/20190215_텍스트마이닝 자체 회의/20190215_텍스트마이닝 자체 회의/뉴스 데이터 추출/분석/통합(2014-2

In [ ]:
regressorGRU.load_weights(filename)
pred = regressorGRU.predict(test_feature)